In [54]:
# @author : Nur Aini Rakhmawati
# sentimen bahasa Indonesia
import pandas as pd
import nltk
from nltk.tag import CRFTagger
import re, math
import numpy
import random

def get_scores(dframe):
    count =0
    totalpositive =0.0
    totalnegative =0.0
    for index,row in dframe.iterrows():
        totalpositive = totalpositive + row['pos']
        totalnegative = totalnegative + row['neg']
        count =count + 1
    if count > 0:
        return((totalpositive-totalnegative)/count)
    else:
        return 0

def remove_hashtag(tweet):
    katas=tweet.split()
    #print(katas)
    katasbersih=""
    for kata in katas:
        
        if  kata.startswith('#') == False:
            katasbersih=katasbersih+' '+kata
    return katasbersih

def removeUrl(tweet):
    doc = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*','',tweet, flags=re.MULTILINE)
    doc = re.sub(r'\b(\w*pic.twitter.com\w*)\b.*','',doc, flags=re.MULTILINE)
    doc = re.sub(r'[\d-]','',doc, flags=re.MULTILINE)
    doc = doc.replace("Rp","")
    return doc


def clean_tweet(tweet):
     return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

def klasfikasibaru(tweet):
    try:
        negatif = ['buruk', 'cebong', 'kampret', 'goblok', 'dungu', 'bobrok']
        postitif = ['setuju', 'bagus', 'dukung', '']

        print("Nilainya = "+str(tweet[2]))
        sentimen = ['negatif', 'positif', 'netral']
        if str(tweet[2]) == 'nan':
            return sentimen[random.randint(0,2)]
        elif tweet[2] is not None:
            return tweet[2]
        elif any(x in tweet[5] for x in negatif):
            return 'negatif'
        elif any(x in tweet[5] for x in postitif):
            return 'positif'
        else:
            return 'netral'
    except:
        return 'netral'

def cleanAllTweet(tweet):
    try:
        doc = remove_hashtag(tweet)
        doc = removeUrl(doc)
        doc = clean_tweet(doc)
        print("Clean tweet = "+str(doc))
        return doc
    except:
        pass

def get_sentimen(doc):
    ct = CRFTagger()
    ct.set_model_file('D:/all_indo_man_tag_corpus_model.crf.tagger')

    doc=remove_hashtag(doc)
    doc=clean_tweet(doc)
    #print(doc)
    #pisah perkalimat
    sentences = nltk.sent_tokenize(doc)
    #pisah per kata
    stokens = [nltk.word_tokenize(sent) for sent in sentences]
    #tag indonesia
    taggedlist = ct.tag_sents(stokens)
    #print(taggedlist)

    #sentiword Indonesia
    barasa=pd.read_csv('D:/barasa-ID.txt',delimiter='\t',encoding='utf-8',header=0, names=['syn', 'goodness', 'lemma', 'pos','neg','dummy'])
    score_list=[]
    negasi=""
    for idx,taggedsent in enumerate(taggedlist):
        score_list.append([])
        for idx2,t in enumerate(taggedsent):
            newtag=''
           
            if t[1].startswith('NN'):
                newtag='n'
            elif t[1].startswith('JJ'):
                newtag='a'
            elif t[1].startswith('VB'):
                newtag='v'
            elif t[1].startswith('R'):
                newtag='r'
            elif t[1].startswith('NEG'):
                negasi=t[0]
            else:
                newtag=''       
            if(newtag!=''):    
                if(negasi!=""):
                    kalimat=negasi + ' ' + t[0]
                    negasi=""
                else:
                    kalimat=t[0]
                lemmas=barasa[barasa['lemma'].str.contains(kalimat,na=False)]
                score_list[idx].append(get_scores(lemmas))
    #sentence_sentiment=[]
    totalscore=0.0
    for score_sent in score_list:
        scoresentnow=sum([word_score for word_score in score_sent])/len(score_sent)
        #sentence_sentiment.append(score_sent)
        totalscore=scoresentnow+totalscore
    return(totalscore/len(score_list))


if __name__ == '__main__':
    import nltk
    nltk.download('punkt')

    #doc="@kamu Maaf saya mau menyampaikan layanan farmasi umum di rsud dr soetomo tidak maksimal karena pegawai sedikit waktu ditinggal sholat http://yes.com, antriannya menumpuk sampai 2 jam lebih hanya buat satu macam obat. Yang mengeluh bukan saya saja tapi banyak. Mohon penanganannya."
    #print(get_sentimen(doc))

    #doc = "cth step utk bangun indonesia masuk akal praktik bukti jokowi invest infrastruktur fasilitas sat era ekonomi ga lancar kirim sayur desa jala ga aspal"

    doc = "Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara ngutang, setiap negara itu pasti ngutang,  utang bisa dibayar kalo negara dapet penghasilan. Penghasilan negara itu ya dari pajak"
    print("Nilai senrimen = "+ doc +str(get_sentimen(doc)))

#baru=remove_hashtag("tes123 #halo !! pa ya httP://mau \n \r wawa")
#print(baru)
#print(clean_tweet(baru))
    # tweet = "Bantu majukan perekonomian bangsa bersama Pak Jokowi, yuk! https://twitter.com/BKNSquare/status/1113655944955588610Â â€ #Jokowi"
    #tweet = "Memang cebong tidak bisa kita menghendaki harga cabai sampai Rp 10.000, harga bawang sampai Rp 5.000, petaninya yang kasihan, kata Jokowi #2019JokowiKyaiMaruf #1PeriodeLagiUntukJokowi #PesonaBonoSurfing  https://twitter.com/tag_nusantara/status/1061795598209232896/photo/1 pic.twitter.com/NApI0SoReiÂ"
    #cleanTweet = klasfikasibaru(tweet)
    #print(cleanTweet)
    #for i in range(5):
        #print(str(random.randint(0,2)))




[nltk_data] Downloading package punkt to C:\Users\Windows
[nltk_data]     10\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Nilai senrimen = Negara kita ngutang buat bngun infrastruktur yang udah dipake masyarakat, terus masyarakatnya ngeluh karena negara ngutang, setiap negara itu pasti ngutang,  utang bisa dibayar kalo negara dapet penghasilan. Penghasilan negara itu ya dari pajak-0.014243356271683064


In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import hstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


tweets = pd.read_csv(r"D:\uts\tweetclean600.csv",delimiter=';', encoding='utf-8')
tweets=tweets.dropna()
tweets=tweets[tweets.Sentimen != '-']
list(tweets.columns.values)

sentiment_counts = tweets.Sentimen.value_counts()

print(sentiment_counts)

from nltk.probability import FreqDist

fdist = FreqDist(tweets[(tweets.Sentimen == 'negatif')])
print(fdist.most_common(50))


#count_vectorizer = CountVectorizer(ngram_range=(1,2))

feature_extraction = TfidfVectorizer()
vectorized_data = feature_extraction.fit_transform(tweets.clean_tweet)
indexed_data = hstack((np.array(range(0,vectorized_data.shape[0]))[:,None], vectorized_data))



def sentiment2target(sentiment):
    return {
        'negatif': 0,
        'netral': 1,
        'positif' : 2
    }[sentiment]

#targets = tweets.Sentimen.apply(sentiment2target)
targets = tweets.Sentimen

from sklearn.model_selection import train_test_split
data_train, data_test, targets_train, targets_test = train_test_split(indexed_data, targets, test_size=0.4, random_state=0)
data_train_index = data_train[:,0]
data_train = data_train[:,1:]
data_test_index = data_test[:,0]
data_test = data_test[:,1:]

from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(svm.SVC(gamma=0.01, C=100., probability=True, class_weight='balanced', kernel='rbf'))
clf_output = clf.fit(data_train, targets_train)

print(clf.score(data_test, targets_test))

y_pred = clf.predict(data_test)
print(y_pred)
print("Accuracy:",accuracy_score(targets_test, y_pred))
print(recall_score(targets_test, y_pred, average='weighted'))
print(precision_score(targets_test, y_pred, average='weighted'))
print(f1_score(targets_test, y_pred, average='weighted'))

sentences = feature_extraction.transform([
    "pantes kampret orang goblok coba belajar pakar pakar ekonomi kasih pemaparan biar tau gimana berbisnis",
    
])
print(clf.predict(sentences))
print(clf.predict_proba(sentences))



D:\python\anaconda\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


2    612
1    607
0    596
Name: Sentimen, dtype: int64
[('id', 1), ('Sentimen', 1), ('clean_tweet', 1)]
0.5922865013774105
[0 1 1 1 2 2 0 0 1 2 1 2 0 2 2 0 1 2 1 0 2 2 0 2 0 2 2 2 2 1 2 1 2 0 2 2 0
 0 0 0 1 0 0 2 2 0 1 0 1 0 2 1 1 0 2 2 2 0 2 2 2 0 0 1 0 2 0 0 0 2 0 1 1 0
 2 1 2 0 2 2 1 0 0 0 1 2 2 1 2 2 1 2 2 0 2 1 2 1 1 0 0 0 2 0 1 0 1 0 1 0 1
 0 1 2 1 0 2 1 0 0 0 2 1 2 1 1 0 1 1 0 2 2 2 1 0 2 0 0 0 2 1 1 2 2 1 0 1 1
 2 1 0 2 0 0 0 1 0 1 1 2 0 1 2 2 2 0 0 1 1 1 2 2 0 1 2 2 0 2 0 2 2 1 1 2 0
 1 2 0 0 1 0 0 0 0 0 0 0 0 2 1 2 0 0 2 0 2 0 1 0 0 1 0 0 2 2 2 0 0 1 2 1 0
 2 2 2 1 2 1 1 0 1 0 0 0 1 1 0 0 2 1 2 1 1 0 0 2 2 1 0 2 0 1 2 1 0 0 2 2 0
 2 2 0 1 2 1 0 1 1 1 2 2 0 1 2 0 0 2 1 0 0 0 1 1 0 2 1 0 2 2 1 2 1 2 2 1 2
 2 1 1 0 2 2 0 2 0 1 0 2 1 1 2 1 0 0 2 1 2 2 1 0 1 0 0 2 0 0 1 1 0 2 1 2 0
 2 1 2 2 0 0 2 1 0 2 1 1 0 1 1 1 1 0 1 0 1 2 1 0 1 2 1 0 0 2 1 2 2 0 2 0 2
 0 2 2 2 0 2 1 1 0 0 1 0 1 1 0 2 2 0 2 1 2 2 2 0 0 1 2 1 1 0 0 0 1 0 2 2 1
 1 2 0 0 1 0 2 0 1 2 2 2 1 1 1 0 2 1 0 1 0 2 2 2 2 

In [ ]:
import pandas as pd
tweets1= pd.read_csv("D:/tweets333-only-withlbscore-new.csv",delimiter='|', encoding='utf-8')
tweets1=tweets1.dropna()
tweets1=tweets1[tweets1.clean_tweet != '-']
tweets1

In [87]:

sentences=tweets1.clean_tweet.tolist()
sentences = feature_extraction.transform(sentences)
for i in range(len(tweets1)):
    if clf.predict(sentences[i])==0:
        print("negatif")
    elif clf.predict(sentences[i])==1:
        print("netral")
    elif clf.predict(sentences[i])==2:
        print("positif")
    print(tweets1.clean_tweet[i],clf.predict(sentences[i]))



positif
 sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat [2]
positif
 tenang msih ngoceh ttp jokowi harga mati [2]
positif
 kerjasa ekonomi bilateral multilateral doakan tps dengarkan suara hati [2]
netral
 pilih pemimpin pencapaiannya pencapaian ekonomi pemerintahan jokowi coblos kerja [1]
negatif
 nginti abdillahtoha ngacaaa ngajarin sandi ekonomi dibanding jokowi [0]
positif
 pakta integritas prabowo sandi ulama terlibat ijtimak ulama pancasila nkri harga mati khilafah khilafah bualan makhluk kolam [2]
negatif
 dengar mbak vidya curup beli lipstik purbasari harga ribu beli lipstik nyx ribu [0]
netral
 assalamu alaikum suara sgt berharga upaya memenangkan merelakan suara jkt ktp daerah semoga suara disertakan terbalaskan doa semoga allah swt mengabulkan aamiin rabb [1]
negatif
 menghargai relawan puasa tirakatan munajad semalam suntuk berdoa kemenangan mhn dibalas tweet kontribusi ngopi rokok tirakat mujahadah gagal tira

positif
 kepemimpinan presiden jokowi indonesia maju perekonomian dunia menentu [2]
netral
 video sudut pandang berbeda video membahas real energi sosial lingkungan ekonomi politik budaya batubara [1]
netral
 presiden jokowi raja salman sepakat tingkatkan kerja ekonomi [1]
negatif
 media asing tunjukkan beranganangan lantas menabur fitnah jijik kebohongan kesombongan kau dibanding kau apaapanya samasama makan gaji pembayar pajak kau tidurnya [0]
netral
 dpt harga rb toko abang [1]
netral
 prabowosandi tegaskan ambil gaji teilih [1]
netral
 teilih prabowosandiaga janji ambil gaji [1]
netral
 penutup debat pamungkas prabowosandi mandat seperseun gaji ambil [1]
negatif
 utang prabowo bwjibun utng ln utang pribadinya puluhan triltun gaji diambil busyeettt jngn utang bebas dibayar jemarin dipailitkan biar bayar pemerintah mk bwrambiai mengumbar janji [0]
positif
 janjinya ngehapusin pajak bikin hutang negara nambah mbak pajak negara [2]
positif
 ijin aspirasi teman desanya penghasilan utama

 pemimpin terbaik ekonominya sesuai goal [2]
netral
 coba ngmng gini jaman lulung m topik btw fungsi dprd mitra bidang legislatif pengawas gubernur menjalankan anggaran sbg pembayar pajak lbh seneng anggota dprd garang [1]
positif
 presiden menghargai istrinya marufamin [2]
positif
 semoga jokowers membaca hidayah allah swt teilih prabowosandiaga janji ambil gaji [2]
netral
 beli lalapan abang lalapan presiden jokowi lalapannya presidennya prabowo harganya dalem hati frontal gini gapapa orang berhak pesta demokrasi negeri pengamat [1]
netral
 nilai ekonomi peningkatan pariwisata peningkatan kesempatan kerja peningkatan kesempatan usaha promosi kota penyelenggaraan penciptaan nilai ekonomi diraih negara sebagi penyelenggara asian games jokowi hebat jokowi [1]
netral
 paguyuban suamisuami gaji paspasan keberadaannya nuntut beliin tas hermes [1]
positif
 dampak ekonomi langsung asian games meliputi tahap konstruksi operasional penyelenggaraan kedatangan pengunjung berkontribusi pertumbuha

positif
 tau heran pdhl pemerintahan sblmnya bbm hrga pangan tbtb skrg stabil kisruh emng susah mb cinta mati didukung wlpn wowo ngelakuin kesalahan kg tetep dibela mati pdhl jokowi harga mati [2]
netral
 dukung jokowi dahlan iskan kubu prabowo dukung jokowi miliki harapan pertumbuhan ekonomi terlaksana [1]
positif
 semalem nyinggung mobile legend akuin anak muda doyan ngegame adek cowok kuliah kesehariannya ngegame profesinya penghasilannya gede ngalahin gaji [2]
positif
 dampak ekonomi langsung asian games meliputi tahap konstruksi operasional penyelenggaraan kedatangan pengunjung berkontribusi pertumbuhan pdb nilai ekonomi peningkatan output sektor penyerapan tenaga kerja peningkatan pendapatan riil jokowi hebat [2]
negatif
 ambil gaji perusahaanya menguadai proyek sobat golput tps pilih jokowi selamatkan bangsa [0]
positif
 sandi memikirkan ekonomi orang mengeluh berkunjung jokowi memikirkan ekonomi nusantara [2]
positif
 jokowi kemajuan indonesia inflasi terjaga perbankan sehat pe

KeyboardInterrupt: 

In [89]:
import csv

sentences=tweets1.clean_tweet.tolist()
sentences = feature_extraction.transform(sentences)
csv.register_dialect('myDialect',
delimiter = '|')
with open('D:/predisct_svm2.csv', 'a',encoding='utf-8') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(["number","Sentimen","Atribut_new","tweet","lexicon"])
    x=1
    
    
    for i in range(len(tweets1)):
        if clf.predict(sentences[i])==0:
            sentimen_svm="negatif"
        elif clf.predict(sentences[i])==1:
            sentimen_svm="netral"
        elif clf.predict(sentences[i])==2:
            sentimen_svm="positif"
        print(tweets1.clean_tweet[i],clf.predict(sentences[i]))
        writer.writerow([x,tweets1.sentimen[i],tweets1.clean_tweet[i],tweets1.sentimen_score_lb[i],tweets1.sentimen_new[i],sentimen_svm])
        x=x+1
f.close()

 sumatera selatan asian games berdampak ekonomi langsung diprediksi mencapai triliun indonesia maju jokowi hebat [2]
 tenang msih ngoceh ttp jokowi harga mati [2]
 kerjasa ekonomi bilateral multilateral doakan tps dengarkan suara hati [2]
 pilih pemimpin pencapaiannya pencapaian ekonomi pemerintahan jokowi coblos kerja [1]
 nginti abdillahtoha ngacaaa ngajarin sandi ekonomi dibanding jokowi [0]
 pakta integritas prabowo sandi ulama terlibat ijtimak ulama pancasila nkri harga mati khilafah khilafah bualan makhluk kolam [2]
 dengar mbak vidya curup beli lipstik purbasari harga ribu beli lipstik nyx ribu [0]
 assalamu alaikum suara sgt berharga upaya memenangkan merelakan suara jkt ktp daerah semoga suara disertakan terbalaskan doa semoga allah swt mengabulkan aamiin rabb [1]
 menghargai relawan puasa tirakatan munajad semalam suntuk berdoa kemenangan mhn dibalas tweet kontribusi ngopi rokok tirakat mujahadah gagal tirakat munajad [0]
 prabowo sandi menang otomatis bersedekah kaum farkir 

 pemahaman tata negara pemahaman peraturan pemahaman kebijakan ekonomi pemahaman kursi mentri diisi kompeten ketimbang prabowo jatah kursi menteri paol pendukungnya [0]
 gelar pertemuan bilateral presiden jokowi raja salman sepakat kerja ekonomi ditingkatkan bidang energi pariwisata [1]
 salah arahnya pembangunan ekonomi indonesia terlepas andil menyalahkan kesalahan kesalahan presidenpresiden [1]
 harga tiket pesawat diturunkan brharap aspek trlbh dngan mahal harga tiket pswt terbang mahal tarif biaya expdisi kota melonjak [0]
 die zaman jokowi ekonomi anjlok gadget bayar wifi bilang anjlok n serba mahal [0]
 prabowo berbuat rakyat gaji karyawannya dibayar heheheheee [1]
 akui kekurangan akui prabowo kristen mualaf ibunya kristen hashim kristen sholat wudhu gerakan salah ngaji dipecat tni elit bajingan [0]
 om prabs membangkitkan ekonomi lepas hutang dunia lepas cengkraman elite global kuncinya pakai uang kertas pakai emas perak alat tukar resiko melawan dunia hadapi bangsa [1]
 gaji 

 ketuaku harga partai kinerja jokowi dinikmati hasilnya kemaslahatan ummat garis perjuangan ketua beihak masyarakat lemah jokowi membantu masyarakat desa [2]
 sedih mudahan periode pemerintahan jokowi harga kampret minimal ribu [0]
 mamfaat diperoleh pelaksanaan asian games sebatas bidang pembinaan olahraga nasional sisi pembangunan infrastruktur pariwisata perekonomian luas indonesia maju jokowi hebat [2]
 kelangsungan peningkatan perekonomian kesehatan pendidikan kesejahteraan sosial indonesia jokowi menerapkan ideide dngan harapan faktor indonesia meningkat seiring berjalannya dimasa pmerintahannya [1]
 sandi bahas salah poin buktikan jokowi bagus emang prabowo tau ekonomi makro ekonomi skala nasional [2]
 prabowo salah arahnya pembangunan ekonomi indonesia terlepas andil presiden jokowi [1]
 khusus presiden jokowi pembangunan manusia tantangan global menuntut penguatan sumber daya manusia sdm penopang perekonomian [1]
 bukti pemerataan pembangunan diharapkan koneksi warga papua efi

 era presiden jokowi ekonomi indonesia maju [2]
 langsung kubu oposisi tau presiden jokowi dalm pemerintahanya kebijakan ambil beihak rakyat bbm harga [1]
 membangun merevitalisasi pasar rakyat menjaga stabilitas harga ketersediaan stok bahan pokok menyerap produksi negeri meningkatkan ekspor mandat presiden jokowi [1]
 pencapaian perekonomian indonesia dibandingkan negara berkembang pasalnya pertumbuhan ekonomi indonesia era pemerintahan jokowijusuf kalla dinilai [2]
 jokowi fokus bangun infrastruktur kunci pemerataan ekonomi [2]
 gagal naekin ekonomi bilang gagal maaf alesan ngalor ngidul contoh prabowo salah bikin gaduh maaf susahnya nyebut huruf maaf ekonomi dibawah zaman sby [0]
 komitmen ambil gaji teilih dikemanakan gaji prabowosandi [1]
 prabowo sandiaga janji ambil gaji presiden wapres [1]
 suuzon mba hehe hargai hasil kerja jokowi membangun negri benci emang susah hargai kerja keras orang [2]
 penasaran angkat isu esport jokowi orang kejam banget isu coba pikir dampaknya ekon

 prabowo sandi ambil gaji teilih gajinya diserahkan fakir miskin dhuafa berjanji membersamai orang tua berjuang [2]
 jokowi raja salman sepakat tingkatkan kerja ekonomi via [1]
 jokowi mengimbangi serangan prabosan mikro sengaja menyeret prabosan visi makro dibidang ekonomi digital membahasnya sisi mikro sandi mengalami kesalahan fatal menyebut big data ektp [1]
 prabowo ekonomi indonesia salah arah presiden jokowi kader demokrat koalisi [1]
 ekonomi salah kebangkitan bangsa teruskan perjuagan prabowo sandi padang lawas sumattra utara rajuddin harahap prabowosandi a dukugan presiden wakil presiden amin [2]
 presiden raja salman menggelar pertemuan bilateral istana pribadi raja alqasr alkhas minggu sore pertemuan presiden raja salman sepakat meningkatkan kerja bidang ekonomi foto berita biro pers setpres [1]
 harga tiketnya turun [1]
 bikin puisi ekonomi persen kayak [0]
 meningkatkan tax ratio ekonomi terguncang ninu ninu [1]
 era jokowi harga bbm papua harga jawa [1]
 trus bayar pajak

 mendorong ekonomi kreatif digital masyarakat booming jokowi ngomong maksud pemikiran sudut pandang kubu sebelah mohon maaf kebuka [2]
 memilih prabowo sandi bersodaqoh gaji prabowo sandi disedekahkan fakir miskin mantapkan sebarkan [2]
 jalan bagus kmna gampang ekonomi smakin membaik bilang infrastuktur giliran jalanan jelek komen [0]
 fadli zon prabowo ingatkan ekonomi indonesia beijak pasal uud [1]
 beda sebelah prabowo sandi ambil gaji presiden fakir miskin [2]
 kampanye akbar gbk pancasila nkri bhinneka tunggal ika uud final harga mati sehingg diganggu gugat [1]
 tax amnesty kemarin prabowo sandiaga uno bayar pajak namanya paradise papers [0]
 malu akui kegagalan mmg janjijanji direalisasi rakyat memilih janji stop impor cabut subsidi buka lapangan kerja ekonomi persen mobil esemka hasilnya nol wajar rakyat pemimpin [2]
 prabowo sandi gaji sumbangkan kaum duafa fakir miskin [2]
 kaderkader partai demokrat walkout prabowo menyalahkan presidenpresiden jokowi bertanggung kesalahan ke

 esport lg buming hobi dipertandingkan dibanyak negara asian games asean games millenial banget bicara pemerataan kesejahteraan calon elite gaji karyawannya terkatung jokowi mempraktekkan [2]
 garis jokowi ekonomi digital dalamnya bisnis games pembuat game pemain game lbh simplenya buka playstore ekonomi digital jokowi mnyebut salah satunya [2]
 santun pertumbuhan ekonomi [1]
 yaelaah jokowi nurunin harga butuh perhitungan cermat matang asalan [2]
 debat malam capres nomor urut perekonomian indonesia salah arah ditandai mengalirnya kekayaan negeri temukan kesesuaian pernyataannya [1]
 namanya wowo menuding sby dibalik kesalahan ekonomi indonesia teguh dengerlah wowo [0]
 rakyat butuh fundamental ekonomi pangan sandang papan digital model mobile legend timeout pencitraan [0]
 esports jokowi prabowo sandiaga idn times jokowi ekonomi digital esport prabowo [1]
 sayang jokowi membahas ekonomi digital kegunaan multifungi ektp [0]
 prabowo harga petani harga konsumen murah menutupi kerugian 

 gaji karyawan belom dibayar prabowo dibayar pake [0]
 kalimat penutup debat mengambil gaji serupiah diinfaqkan kaum du afa anak yatim [1]
 dasar ego menghargai wakilnya lihat prabowo sandi melengkapi [2]
 mencoblos prabowo sandi membantu beramal beliau gajinnya kaum dhuafa anak yatim membutuhkan [2]
 bersedekah mencoblos beramal gaji anak yatim dhuafa bersedekah [2]
 mohon bilang pangeran muhammad bin salman tolong direalisasikan beli biar kesusahan bayar gaji beli pemainnya [1]
 hahahaha tema debat perekonomian pikir segi pertanian jual beli sembako hubungannya kondisi perekonomian negara negara agraris paham maksudnya cari maksud negara agraris debat [2]
 jalannya debat jokowi sandiaga uno ibuibu mengeluh hargaharga [1]
 teilih prabowosandi takkan ambil gaji negara segini gaji presiden wapres manado [1]
 prabowosandi paslon pemaparan visi misi pro rakyat hubungan nyapun harmonis dwi tunggal menyumbangkan gajinya maklum tajir [1]
 maksud sombong menghidupi keluarga kartu pabowosandi 

 uno prabowo komitmen gaji ambil hak semoga amanah lupa salam jari salam jak mania [2]
 hak penuh pejabat gajinya boss tunjangan peruntukannya dipakai sembarangan [1]
 gaji tunjangan struktur gaji pejabat indonesia ajaib minim banget tunggangan fasilitas bejibun wan gabener berani melepas tunjangan rakyat [0]
 coblos prabowo sandi pahala sedekah gaji prabowo sandi biberikan kwpada anak yatim dhuafa [2]
 teman sejati amal bilang memilih prabowo sandi amal gaji beliau amal [2]
 langsung om wowo mengapresiasi perekonomian negara dibawah kepemimpinan jokowi terima kasih om jokowi [2]
 om wowo kesalahan jokowi kesalahan pemimpinpemimpin mengelola perekonomian negara alhamdulillah [2]
 prabowosandi ambil gaji netizen subhanallah pemimpin sejati via [1]
 bismillah rabu april mencoblos prabowo sandi samahalnya beramal gaji beliau disumbangkan dhuafa [2]
 silahkan pilih prabowo sandi ambil gaji sepeser teilih [1]
 prabowo emang prestasi pake logika mikir beda ma pendukung walkot solo dah dp pen

 kasih statistik penghasilan pro gamer ngelebihi gaji seumur hidup amda [0]
 dinilai membuktikan komitmennya meningkatkan perekonomian bidang industri [2]
 masya allah tearssss allah bless sir and gaji diterima allah memberimu indonesia [2]
 debat bahas esport caper milenial emang esport lapangan kerja kartu prakerja gajinya setau lapangan kerja youtubers esport [0]
 masya allah komitmen mengambil gaji menjabat [2]
 iyalah gaji diambil [2]
 bersyukur gaji rakyat tusuk allah maha membolak balikkan hati [2]
 ambil gaji jae disuruh cuti ogah pake fasilitas negara [0]
 closing mantul masya allah beliau berdua berkomitmen menang mengambil gaji sepeseun beliau serahkan kaum dhuafa allahu akbar allahu akbar [2]
 prabowosandi sepakat mengambil gaji teilih berjanji menghormati ulama mematuhi fakta integritas ijtima ulama [1]
 woow gaji prabowo sandi diambil disumbangkan teilih masyaallah [2]
 pengabdian negara keikhlasan berdua lillahita ikhlas ngomong ikhlas mengabdi menerima gaji sepersenpun 

In [25]:
import pandas as pd
tweets2= pd.read_csv("D:/predisct_svm2.csv",delimiter=';', encoding='utf-8')
TP_negatif=0
TP_netral=0
TP_Positif=0
z=0
y=0
x=0
for i in range(len(tweets2)):
    if str(tweets2.attribute_new[i]) =="ekonomi" and str(tweets2.sentimenSvm[i])=="positif":
        x=x+1
    elif str(tweets2.attribute_new[i]) =="ekonomi" and str(tweets2.sentimenSvm[i])=="netral":
        y=y+1
    elif str(tweets2.attribute_new[i]) =="ekonomi" and str(tweets2.sentimenSvm[i]) =="negatif":
        z=z+1
print(x,y,z)

123 118 109


In [14]:
import pandas as pd
tweets2= pd.read_csv("D:/predisct_svm2.csv",delimiter=';', encoding='utf-8')
tweets2.sentimenSvm

0       positif
1       positif
2       positif
3        netral
4       negatif
5       positif
6       negatif
7        netral
8       negatif
9       positif
10      negatif
11      negatif
12      positif
13       netral
14      positif
15       netral
16      positif
17       netral
18      positif
19       netral
20       netral
21      positif
22      negatif
23      positif
24       netral
25      positif
26       netral
27      negatif
28      positif
29       netral
         ...   
1020    positif
1021    positif
1022    positif
1023    negatif
1024    positif
1025     netral
1026    positif
1027    positif
1028    positif
1029    positif
1030    positif
1031    positif
1032    negatif
1033    positif
1034     netral
1035    positif
1036     netral
1037    positif
1038    negatif
1039     netral
1040    positif
1041     netral
1042    negatif
1043    positif
1044     netral
1045     netral
1046     netral
1047    negatif
1048     netral
1049     netral
Name: sentimenSvm, Lengt

In [161]:
tweets2.sentimenLexicon

AttributeError: 'DataFrame' object has no attribute 'sentimenLexicon'

In [183]:
import pandas as pd
tweets21= pd.read_csv("D:/predisct_svm2.csv",delimiter='|', encoding='utf-8')
tweets21x =  tweets21['sentimenLex ']
tweets21x

0        netral
1        netral
2        netral
3       positif
4       positif
5        netral
6        netral
7       positif
8        netral
9       positif
10      positif
11       netral
12       netral
13       netral
14       netral
15      positif
16      negatif
17       netral
18      positif
19      negatif
20       netral
21       netral
22      negatif
23      positif
24      positif
25      positif
26       netral
27       netral
28       netral
29      negatif
         ...   
1020    negatif
1021     netral
1022    negatif
1023     netral
1024    negatif
1025    positif
1026     netral
1027    positif
1028     netral
1029    positif
1030    positif
1031     netral
1032    negatif
1033     netral
1034    negatif
1035    positif
1036     netral
1037    positif
1038     netral
1039     netral
1040     netral
1041     netral
1042     netral
1043     netral
1044    negatif
1045    positif
1046    negatif
1047    negatif
1048    negatif
1049     netral
Name: sentimenLex , Leng

In [179]:
tweets2.lexicon

AttributeError: 'DataFrame' object has no attribute 'lexicon'

In [186]:
 barasa=pd.read_csv('D:/barasa-ID.txt',delimiter='\t',encoding='utf-8',header=0, names=['syn', 'goodness', 'lemma', 'pos','neg','dummy'])
print(barasa)

           syn goodness                       lemma    pos  neg  dummy
00001740-a   B        L               berdaya upaya  0.125  0.0    NaN
00001740-a   B        L                berkemampuan  0.125  0.0    NaN
00001740-a   B        L              berkesanggupan  0.125  0.0    NaN
00001740-a   B        L                berkeupayaan  0.125  0.0    NaN
00001740-a   B        L                     beroleh  0.125  0.0    NaN
00001740-a   B        L                       boleh  0.125  0.0    NaN
00001740-a   B        L                       cakap  0.125  0.0    NaN
00001740-a   B        L                       cekap  0.125  0.0    NaN
00001740-a   B        L                      handal  0.125  0.0    NaN
00001740-a   B        L                    jurukira  0.125  0.0    NaN
00001740-a   B        L                        kuat  0.125  0.0    NaN
00001740-a   B        L                        lalu  0.125  0.0    NaN
00001740-a   B        L                      lantas  0.125  0.0    NaN
000017